### Typical iris dataset using a simple imputer and a standard scaler before splitting and training

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

data = load_iris(as_frame=True)
X, y = data.data, data.target

# Added some missing values
import numpy as np
X.iloc[::10, 0] = np.nan

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

model = LogisticRegression()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))

**Task 1:** fit the steps into a pipeline.



**Task 2:** Use a different model but the same pipeline.

**Bonus Task:** Create a custom transformer to add Petal Area to the dataset.